In [83]:
from dotenv import load_dotenv
import serpapi
import os, requests
from youtube_transcript_api import YouTubeTranscriptApi
from google import genai

load_dotenv()

True

In [84]:
def change_youtube_link_to_video_id(link: str):
    return link.replace("https://www.youtube.com/watch?v=", "")

def get_related_youtube_searches(query):
    client = serpapi.Client(api_key=os.getenv('SERPAPI_API_KEY'))

    results = client.search({
        'search_query': query,
        'engine': 'youtube'
    })

    youtube_links = [(video['title'], change_youtube_link_to_video_id(video['link'])) for video in results['video_results']]
    return youtube_links[:1]

def format_transcripts(video_data: tuple[str, str]):
    ytt_api = YouTubeTranscriptApi()

    youtube_transcripts = []
    for title, video_id in video_data:
        transcripts = ytt_api.fetch(video_id)
        transcript_text = "\n".join([transcript.text for transcript in transcripts])

        youtube_transcripts.append(f'''Title: {title}\nTranscript:\n {transcript_text}\n''')
    
    return youtube_transcripts

def get_youtube_transcripts(query: str) -> list[str]:
    """Fetches YouTube transcripts for differnt topics.

    Args:
        query: Topic of interest

    Returns:
        A list containing the transcripts of different youtube videos.
    """
    video_data = get_related_youtube_searches(query)

    return format_transcripts(video_data)

In [85]:
transcripts = get_youtube_transcripts('Jimmy Carter')

In [86]:
transcripts[0]

"Title: The unlikely life story of Jimmy Carter\nTranscript:\n they honored late President Jimmy\xa0\nCarter in his home state of Georgia\nyesterday a motorcade with Carter's flag draped\xa0\ncasket traveled through his hometown of plains and\xa0\xa0\nthen onto Atlanta where he Now lies in Repose at\xa0\nthe Carter presidential center before ceremonies\xa0\xa0\nmove on to Washington on Tuesday we've asked Rita\xa0\nbraver to take a closer look at the unlikely life\xa0\xa0\nstory of our 39th president I would not tell a\xa0\nlie I would not mislead the American people and\xa0\xa0\nI would not betray your trust if anything Define\xa0\nJimmy carter it was his basic decency and morality\xa0\xa0\nthank you again please thank you everybody\xa0\nelected in 1976 in the aftermath of Watergate\xa0\xa0\nand Gerald Ford's controversial pardon of Richard\xa0\nNixon Carter believed he had a responsibility to\xa0\xa0\nrestore Americans' faith in the Integrity of their\xa0\ngovernment we want to be pr

In [89]:
config = {
    'tools': [get_youtube_transcripts],
}

client = genai.Client(api_key=str(os.getenv('GEMINI_API_KEY')))

response = client.models.generate_content(
    model='gemini-1.5-flash', 
    config = config,
    contents='Jimmy carter'
)
print(response.text)

The response is a JSON object containing a list of YouTube transcripts about Jimmy Carter.  The first transcript discusses his life, presidency, and post-presidency work.



In [88]:
functions = [
    {
        "name": "get_youtube_transcripts",
        "description": "Perform a Google search using SerpAPI and return results",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "Search query"},
                "num_results": {"type": "integer", "description": "Number of results to return"}
            },
            "required": ["query"]
        }
    }
]